### Weber Vulgata: lemmatizzazione

Questo notebook implementa una procedura di lemmatizzazione del testo della Vulgata.  
<ol>
    <LI>Il testo della Vulgata è ottenuto facendo il parsing dei file <a href="https://en.wikipedia.org/wiki/Open_Scripture_Information_Standard">Open Scripture Information Standard (OSIS)</a> presenti nella <a href="https://drive.google.com/drive/folders/1KEFTwR1kLz1Ec_-vU3k-yoFcaynBoP7x">cartella</a> del drive condiviso del WP8. 
    <br>I file OSIS vengono processati nell'ordine definito nel file <b>config.yaml</b> che deve essere posizionato nella stessa directory del nb, il risultato di questa fase è un dataframe in cui cisacuna riga contiene il testo di un versetto e le informazioni di localizzazione.</LI>
    <li> I lemmi sono ottenuti tramite il Web Service <a href="https://lila-erc.eu/LiLaTextLinker/"> LiLa TextLinker</a>.  <br>
         Il testo di ogni versetto viene processato dal TextLinker e i lemmi individuati vengono associati a ciascuna parola del testo.  
    </li>
</ol>

####  Librerie e configurazione

In [1]:
import pandas as pd
import os
import yaml
from itserr_lib import OsisParser as op

In [2]:
if (os.path.isfile('config.yaml')):
    configfile="config.yaml"
else:
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), "config.yaml")
try:
    with open(configfile, 'r') as stream:
        try:
            conf=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
except FileNotFoundError:
    print('Warning config.yaml file not present! Please create it and set the values, store it in the main directory')

#### 1 Creo il dataframe con il testo della Vulgata

In [3]:
files=conf['TEXTFILES']
source=conf['LIBRI_Vulgata']

In [4]:
opars=op()

In [ ]:
df_opera=opars.getDataFrame(source,files)

100%|██████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 84.25it/s]


Creating IRIs...


#### Controlli

In [ ]:
# Versetti con numero duplicato
df_opera[df_opera.duplicated(['idverso'], keep=False)].sort_values(['idverso']).head(10)

In [ ]:
# salti nella numerazione dei versetti
parti=df_opera.libro.unique()
tot=0
for mpa in parti:
    mt=df_opera[df_opera.libro==mpa]
    capitoli=mt.numcap.unique()
    for mcap in capitoli:
        mtvers = mt[mt.numcap==mcap]
        col_one_list = mtvers['numverso'].tolist()
        a = col_one_list[0]
        b = col_one_list[-1]
        if(a>1):
            a=1
        nums = set(col_one_list)
        expected = [f"{str(i)}" if i in nums else 'NA' for i in range(a,b+1)]
        if ('NA' in expected):
            tot+=1
            print('---')
            print(f'libro: {mpa}, capitolo: {mcap}, {expected}', end='\n')
        else:
            if (len(expected)!=len(nums)):
                print(f'OK {mcap}: {expected}')

#### 2 Associo alle parole della Vulgata i lemmi, usando TextLinker di LiLa

#### Carico i lemmi

Le chiamate al TextLinker per il totale dei versi dell'opera richiede molto tempo, il lavoro è stato fatto ed il risultato è statro salvato come file JSON.

In [ ]:
# ESEMPIO: lemmatizzazione dei primi due versetti della Genesi
test=opars.getLemmi(df_opera[0:2])

In [ ]:
test.head()

Caricamento dei lemmi dal file JSON

In [ ]:
df_lemmi = pd.read_json(path_or_buf='data/output/lemmi_1.json', orient='table')
df_lemmi.info()

In [ ]:
len(df_lemmi.token.unique())

In [ ]:
df_lemmi.head()